# Multi-Agent Systems (MAS) in ADK — A Crisp, Visual Guide

This module explains how to structure agentic applications in **ADK** using **multiple agents** instead of one monolith. You’ll learn the core composition primitives, common patterns, and when to use each — with diagrams first, minimal code second.

_References:_ concise concepts adapted from the official ADK docs (Multi-Agents section) and reinforced with the provided transcript snippets where they match the ideas (citations inline).

## Learning goals
- Understand **why** to use a Multi-Agent System (MAS) in ADK.
- Learn the **primitives**: hierarchy, workflow agents (Sequential / Parallel / Loop), and interaction mechanisms.
- Recognize and sketch **common patterns**: Coordinator/Dispatcher, Sequential Pipeline, Parallel Fan-Out/Gather, Hierarchy/Decomposition, Generator-Critic, Iterative Refinement, Human-in-the-Loop.
- See a **minimal runnable skeleton** and how to run with **ADK Web**.

> **Why MAS?** Modularity, specialization, reuse, clearer control-flow (via workflow agents), and simpler reasoning about complex apps.

## What is a Multi-Agent System in ADK?
An MAS is an application where **multiple agents** (LLM agents, workflow agents, or custom agents) collaborate — often in a **hierarchy** — to achieve a larger goal. This is highlighted in the transcript’s walkthrough of composing agents and wrapping them in workflows (e.g., parallel inside sequential, with a summarizer combining state).

```mermaid
flowchart TD
  A[Parent Agent] --> B[Specialist A]
  A --> C[Specialist B]
  A --> D[Workflow Agent]
  D --> E[Step 1]
  D --> F[Step 2]
```

**Single-parent rule:** an agent instance can only belong to **one parent** (helps keep the tree unambiguous).

**Root agent:** all user requests enter the system via a root agent (reinforced in transcript when introducing first agent and run flows).

## ADK Primitives for Composition

### 1) Agent hierarchy (parent → sub_agents)
- Build a tree by passing `sub_agents=[...]` to the parent.
- ADK sets each child’s `parent_agent` automatically.
- The hierarchy defines the scope for **LLM-driven delegation** and for **workflow agents** to orchestrate their children.

```python
# Concept only (minimal):
from google.adk.agents import LlmAgent

greeter = LlmAgent(name="Greeter", model="gemini-2.0-flash-001")
helper  = LlmAgent(name="Helper",  model="gemini-2.0-flash-001")

coordinator = LlmAgent(
    name="Coordinator",
    model="gemini-2.0-flash-001",
    description="Routes and coordinates tasks",
    sub_agents=[greeter, helper]
)
```

```mermaid
flowchart TB
  C[Coordinator]:::parent --> G[Greeter]
  C --> H[Helper]
  classDef parent fill:#eef,stroke:#88a;
```

### 2) Workflow agents as orchestrators
Workflow agents don’t do the work themselves; they **control the flow** of their `sub_agents`.

**a) SequentialAgent** — runs children **in order**, sharing the same context/state. The transcript describes wrapping work and then handing off to a synthesizer that reads state.

```mermaid
flowchart LR
  S[SequentialAgent] --> A[Step A]
  A --> B[Step B]
  B --> C[Step C]
```

**b) ParallelAgent** — runs children **concurrently**; they write to shared state under distinct keys (see transcript on parallel agents and then combining results).

```mermaid
flowchart TB
  P[ParallelAgent]
  P --> P1[Worker 1]
  P --> P2[Worker 2]
  P --> P3[Worker 3]
```

**c) LoopAgent** — repeats a mini-workflow until a stop condition (e.g., `max_iterations` or an agent escalates). Useful for iterative refinement.

```mermaid
flowchart LR
  L[LoopAgent] --> X[Step]
  X --> |check| Y{Stop?}
  Y -- no --> L
  Y -- yes --> Z[Done]
```

### 3) Interaction & communication mechanisms

**a) Shared session state** — Agents read/write `state` to pass data (e.g., via `output_key`). The transcript repeatedly emphasizes agents **saving to state** then a later agent **synthesizing** a report.

**b) LLM-driven delegation (transfer)** — An LLM agent can **delegate** to a sub-agent by calling a transfer function the framework recognizes (scope is the hierarchy).

```mermaid
sequenceDiagram
  participant User
  participant Parent as Parent LLM Agent
  participant Child as Target Sub-Agent
  User->>Parent: Request
  Parent->>Parent: LLM decides: transfer_to_agent("Child")
  Parent->>Child: Invoke with shared context
  Child-->>Parent: Response + state updates
  Parent-->>User: Final answer
```

**c) Explicit invocation (AgentTool)** — Wrap an agent as a **tool** the parent can call synchronously; results and state propagate back up.

```mermaid
flowchart LR
  A[Parent LLM Agent] -- calls --> T[[AgentTool: Child Agent]]
  T --> R[Child runs\nupdates state/returns]
  R --> A
```

## Common Multi-Agent Patterns (with when-to-use notes)

### 1) Coordinator / Dispatcher
- **Structure:** One LLM **Coordinator** routes to specialists.
- **Use when:** Requests span distinct domains; you want simple routing rules.
- **Mechanisms:** Hierarchy + LLM delegation **or** AgentTool.

```mermaid
flowchart TB
  R[Coordinator] --> S1[Billing Agent]
  R --> S2[Support Agent]
  R --> S3[Search Agent]
```

### 2) Sequential Pipeline
- **Structure:** `SequentialAgent([A, B, C])`.
- **Use when:** Deterministic multi-step flow; each step consumes prior outputs.
- **Mechanisms:** Shared state (`output_key` → read later). Reinforced in transcript: sequentially wrap work then synthesize.

```mermaid
flowchart LR
  A[Validate] --> B[Process] --> C[Synthesize]
```

### 3) Parallel Fan-Out / Gather
- **Structure:** `ParallelAgent` runs independent tasks; a later step aggregates.
- **Use when:** Independent fetch/compute tasks can run concurrently to reduce latency.
- **Mechanisms:** Distinct state keys per child; subsequent step reads them.

```mermaid
flowchart TB
  P[ParallelAgent]
  P --> A1[Fetch A]
  P --> A2[Fetch B]
  P --> A3[Fetch C]
  A1 & A2 & A3 --> S[Synthesizer]
```

### 4) Hierarchical Task Decomposition
- **Structure:** Multi-level tree; higher levels split goals and assign sub-tasks.
- **Use when:** Complex problems benefit from structured breakdown.
- **Mechanisms:** Delegation across hierarchy; results bubble up.

```mermaid
flowchart TB
  H[Planner] --> H1[Research Assistant]
  H --> H2[Analyzer]
  H1 --> H1a[Web Search]
  H1 --> H1b[Summarizer]
```

### 5) Generator–Critic (Review/Critique)
- **Structure:** `SequentialAgent([Generator, Reviewer])`.
- **Use when:** You want a quality gate or factual check.
- **Mechanisms:** Generator writes to state; Critic reads it and approves/rejects.

```mermaid
flowchart LR
  G[Generate Draft] --> C[Review / Critique]
  C -->|feedback| G
```

### 6) Iterative Refinement (Loop)
- **Structure:** `LoopAgent` wraps steps to refine output.
- **Use when:** You need multiple passes; stop via max iterations or criteria.
- **Mechanisms:** Shared state across iterations; optionally escalate to stop.

```mermaid
flowchart LR
  L[LoopAgent] --> R1[Refine]
  R1 --> K{Meets Criteria?}
  K -- yes --> D[Done]
  K -- no --> L
```

### 7) Human-in-the-Loop (HITL)
- **Structure:** Pause for human approval or input mid-flow.
- **Use when:** Safety, compliance, or external judgment is required.
- **Mechanisms:** Custom tool to request/await human decision; continue based on response.

```mermaid
sequenceDiagram
  participant A as Agent
  participant H as Human System
  A->>H: Send approval request
  H-->>A: Approve / Reject
  A->>A: Proceed based on decision
```

## Minimal runnable skeleton (concept)
> Keep it tiny: one coordinator + two specialists. You can swap real instructions/tools later.

```python
from google.adk.agents import LlmAgent, SequentialAgent, ParallelAgent

# Specialists
fetch = LlmAgent(name="Fetcher",   model="gemini-2.0-flash-001", instruction="Collect short facts.", output_key="facts")
write = LlmAgent(name="Writer",    model="gemini-2.0-flash-001", instruction="Draft a summary from {facts}.")

# A small pipeline: fetch then write
pipeline = SequentialAgent(name="Pipeline", sub_agents=[fetch, write])

# Coordinator that *could* delegate or just run the pipeline
coordinator = LlmAgent(
    name="Coordinator",
    model="gemini-2.0-flash-001",
    description="Coordinates simple 2-step flow.",
    sub_agents=[pipeline]
)
```

**Run with ADK Web** (from your project directory):

```bash
adk web
```

The transcript describes using **ADK Web** to inspect events/state and chat with agents in a browser.

## Best practices & gotchas
- **Clear descriptions** per agent — helps LLM routing/transfer.
- **Single parent** per agent instance — avoid accidental reuse.
- **State keys**: choose distinct keys for parallel workers to avoid collisions.
- **Sequential vs Parallel**: prefer **Parallel** for independent IO; **Sequential** when outputs chain.
- **Loop termination**: set `max_iterations` or use a checker that **escalates** when done.
- **Folder hygiene** (from transcript tips): keep each agent and its tools tidy (e.g., `agent.py`, `tools.py`).
- **Observe with ADK Web**: use UI to trace events, see state, and debug flows.

## Appendix — Quick visual recaps

**Parallel inside Sequential (from transcript narrative):**

```mermaid
flowchart TB
    SQ[SequentialAgent]
    SQ --> PR[ParallelAgent]
    PR --> W1[Worker A]
    PR --> W2[Worker B]
    PR --> W3[Worker C]
    SQ --> SY[Synthesizer<br/>reads shared state]
```

_This mirrors the idea of running multiple agents in parallel, saving to state, then a synthesizer combines them_.

## Sources (for your traceability)
- Transcript references showing **Sequential + Parallel** composition and **ADK Web** usage:
  - Parallel then synthesize inside Sequential, additional emphasis.
  - ADK Web overview and why it’s useful for events/state.
  - Root agent entry concept when building first agent.

> For full API details and the dedicated Multi-Agents page, consult the official ADK documentation you shared alongside this course.